## Simulation of a QKD process invaded by Quantum Cloning Machine

In [57]:
import numpy as np
import pandas as pd

from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI

In [58]:
print('Enter mail address')
email = input()

print('Enter password')
password = getpass()

server_url = r'https://api.quantum-inspire.com'
authentication = BasicAuthentication(email, password)
qi = QuantumInspireAPI(server_url, authentication)

Enter mail address
y.zhang-108@student.tudelft.nl
Enter password
········


In [60]:
qi.list_backend_types()

Backend type: Starmon-5, number of qubits: 5
Backend type: QX single-node simulator, number of qubits: 26
Backend type: QX single-node simulator SurfSara, number of qubits: 31
Backend type: Spin-2, number of qubits: 2


### BB84 protocol: Charles H. Bennett and Gilles Brassard (1984)

Bob generate the key and measurement bases

In [202]:
n_key=64
bitstring = np.random.randint(2,size=n_key)
bases_A=np.random.randint(2,size=n_key)

bases_B=np.random.randint(2,size=n_key)
bases_E=bases_B.copy() # assume quantum memory

The expection key length is 1/2 of the random bitstring.

In [203]:
sum(bases_A==bases_B)

33

If base is 0, measure in Z, otherwise measure in X.

(key,base): state

00: $|0\rangle$ 
10: $|1\rangle$ 
01: $|+\rangle$ 
11: $|-\rangle$ 


Alice prepare the qubit and send it to Bob.

Assume Eve have a Cloning Machine and a Quantum Memory

So she can clone the states and measure in the same base after Bob publish her measurement bases.

In [227]:
def QKD_QACM(key,A_base,B_base,E_base):
    
    # here the qubits used in circuit is 1,3 instead of 0,4, for lower error rate
    
    qasm ='''
    version 2.0

    qubits 5

    # initialize the state
    {0}
    
    #preparation
    Ry q[0], 1.107149
    #rewrite CNOT q[0],q[4] and CNOT q[4],q[0] usign nearest neighbors
    SWAP q[0],q[2] 
    CNOT q[2],q[4]

    Ry q[4], 0.729728
    CNOT q[4],q[2]
    SWAP q[0],q[2]

    Ry q[0], 0.463648

    #copying
    CNOT q[2], q[0]
    CNOT q[2], q[4]
    CNOT q[0], q[2]
    CNOT q[4], q[2]

    #Rotate back and measure
    {1}
    {2}
    '''
    if A_base:
        prep="prep_x q[2]\n"
        if key:
            prep+="    Z q[2]\n"
    else:
        prep="prep_z q[2]\n"
        if key:
            prep+="    X q[2]\n"
            
    meas="measure_x q[2]\n" if B_base else "measure_z q[2]"
    invasion="measure_x q[0]\n" if E_base else "measure_z q[0]"
    
    return qasm.format(prep,meas,invasion)

Check the predetermined subset of their remained key to detect a eavesdropper.

If UQCM used, we can expect the error rate of key is 1/6. 

In [ ]:
N_shots=4096
backend_type = qi.get_backend_type_by_name('Starmon-5')
# backend_type = qi.get_backend_type_by_name('QX single-node simulator')

data=[]
index=0

for param in zip(bitstring,bases_A,bases_B,bases_E):
    qasm=QKD_QACM(*param)
    result = qi.execute_qasm(qasm, backend_type=backend_type, number_of_shots=N_shots)
    hist=result.get('histogram', {}).values()
    data.append(hist)
    index+=1
    
    print(str(index)+"/"+str(n_key))
    print("Execution time: ", result.get('execution_time_in_seconds',{}))
    print("key:{0},bases_Alice:{1},bases_Bob:{2},bases_Eve:{3}".format(*param))
    print(hist)

1/64
Execution time:  0.622592
key:1,bases_Alice:0,bases_Bob:1,bases_Eve:1
odict_values([0.20705689277899345, 0.32029540481400437, 0.17970459518599563, 0.2929431072210066])


In [ ]:
#[bin(int(i))[2:].zfill(5) for i in hist.keys()]
data_sheet=DataFramet=pd.DataFrame(data=np.array(list(map(list,data))), columns=["00","01","10","11"])
data_sheet.head()

In [ ]:
print(QKD_QACM(1,1,1,1))

In [ ]:
np.array(list(map(list,data)))

In [ ]:
data_sheet.to_excel("QKD_Simulation.xlsx")

Find the key by take same measurement bases

In [ ]:
secure_key=key[bases_A==bases_B]

In [ ]:
prob=data_sheet[bases_A==bases_B]

Assume Alice and Bob preserve half of the key for secure check.

Find the probability of Eve get the key without being noticed.

In [225]:
def get_success_prob(bit_string,probs,p):
    '''
    prob: npndarray for probability distribution
    p: number of bit to be check in public channel
    '''
    res=1
    undetected=1
    
    for (k,p) in zip(key[:p],probs[:p]):
        res*=p[k*3] # with 00 or 11
        undetected*=p[k*3]+p[k+1] # k+1=01 or 10
    return res,undetected

In [ ]:
get_success_prob(secure_key,prob.values,4)

### E91 protocol: Artur Ekert (1991)